In [1]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.llms import GooglePalm

## Pre-trained Large Language Model
This LLM will be used as a head in a RAG pipeline to generate natural language fashion answers.

In [2]:
# Get API key for the model
api_file = open("model_API_keys/palm_api.txt", "r")
api_key = api_file.readlines()[1] # read second line
api_key = api_key.strip() # remove newline (\n)

In [3]:
# Pre-trained model
model = GooglePalm(google_api_key=api_key)

/home/sam/anaconda3/envs/QA_llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def get_answer(model, question:str):
    print(f"Question: {question}")
    try:
        answer = model(question)
        print(f"Answer:\n{answer}")
    except:
        print("Similarity search failed.")

In [5]:
# Sample questions
sample_Q = "3rd degree burns on palms, what to do?"
get_answer(model, sample_Q)

sample_Q = "What to do if you are tired?"
get_answer(model, sample_Q)

Question: 3rd degree burns on palms, what to do?
Similarity search failed.
Question: What to do if you are tired?
Answer:
* **Get enough sleep.** This is the most important thing you can do to combat fatigue. Most adults need around 7-8 hours of sleep per night. If you're not getting enough sleep, you're more likely to feel tired during the day.
* **Eat a healthy diet.** Eating a healthy diet can help you maintain your energy levels. Make sure to eat plenty of fruits, vegetables, and whole grains. Avoid processed foods and sugary drinks.
* **Get regular exercise.** Exercise can help improve your mood and energy levels. Aim for at least 30 minutes of moderate-intensity exercise most days of the week.
* **Take breaks throughout the day.** If you're feeling tired, take a break from whatever you're doing and step away for a few minutes. Get some fresh air, stretch, or do something relaxing.
* **Avoid caffeine and alcohol.** Caffeine and alcohol can both interfere with sleep, which can lead

## Load PDF Document
This model converts natural language texts to embedding vectors.

In [6]:
# Load pdf
pdf = os.path.join("data", "GAN.pdf") # PDF file
loader = PyPDFLoader(pdf) # PDF loader
docs = loader.load() # load document

In [7]:
print(f"Number of pages in {os.path.basename(pdf)}: {len(docs)}")
print(f"5th page: {docs[4]}")

Number of pages in GAN.pdf: 9
5th page: page_content='Theorem 1. The global minimum of the virtual training criterion C(G)is achieved if and only if\npg=pdata. At that point, C(G)achieves the value −log 4 .\nProof. Forpg=pdata,D∗\nG(x) =1\n2, (consider Eq. 2). Hence, by inspecting Eq. 4 at D∗\nG(x) =1\n2, we\nﬁndC(G) = log1\n2+ log1\n2=−log 4 . To see that this is the best possible value of C(G), reached\nonly forpg=pdata, observe that\nEx∼pdata[−log 2] + Ex∼pg[−log 2] =−log 4\nand that by subtracting this expression from C(G) =V(D∗\nG,G), we obtain:\nC(G) =−log(4) +KL(\npdata\ued79\ued79\ued79\ued79pdata+pg\n2)\n+KL(\npg\ued79\ued79\ued79\ued79pdata+pg\n2)\n(5)\nwhere KL is the Kullback–Leibler divergence. We recognize in the previous expression the Jensen–\nShannon divergence between the model’s distribution and the data generating process:\nC(G) =−log(4) + 2·JSD (pdata∥pg) (6)\nSince the Jensen–Shannon divergence between two distributions is always non-negative and zero\nonly when t

## Retrieval-Augmented Generation (RAG) Pipeline

In [8]:
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, TokenTextSplitter, NLTKTextSplitter, SpacyTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import Cassandra, Chroma, FAISS # vector database

In [9]:
# Document Splitting: split the document into small chunks
chunk_size = 500
text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0, separator="\n")
chunks = text_splitter.split_documents(docs)

In [10]:
# Pre-trained Text Embedding Model
embedding_model = HuggingFaceInstructEmbeddings

In [11]:
# Embedding vectors
device = "cuda"
query_instruction = "Represent the query for retrieval: "
embeddings = embedding_model(query_instruction=query_instruction,
                             model_kwargs={"device": device},
                            )

load INSTRUCTOR_Transformer
max_seq_length  512


In [12]:
# Vector database (options: Cassandra, Chroma, FAISS, etc.)
db = FAISS.from_documents(documents=chunks, embedding=embeddings)

In [13]:
# vector retrieval from database
search_type = "similarity"
k = 3 # top k similar documents
retriever = db.as_retriever(
    search_type=search_type,
    search_kwargs={"k": k}
    )

In [14]:
# show top k similar texts retrieved from the vector database
similar_texts = retriever.get_relevant_documents("Adversarial modeling")
print(similar_texts[0].page_content)

Generative Adversarial Nets
Ian J. Goodfellow, Jean Pouget-Abadie∗, Mehdi Mirza, Bing Xu, David Warde-Farley,
Sherjil Ozair†, Aaron Courville, Yoshua Bengio‡
D´epartement d’informatique et de recherche op ´erationnelle
Universit ´e de Montr ´eal
Montr ´eal, QC H3C 3J7
Abstract
We propose a new framework for estimating generative models via an adversar-
ial process, in which we simultaneously train two models: a generative model G
that captures the data distribution, and a discriminative model Dthat estimates
the probability that a sample came from the training data rather than G. The train-
ing procedure for Gis to maximize the probability of Dmaking a mistake. This
framework corresponds to a minimax two-player game. In the space of arbitrary
functionsGandD, a unique solution exists, with Grecovering the training data
distribution and Dequal to1
2everywhere. In the case where GandDare deﬁned
by multilayer perceptrons, the entire system can be trained with backpropagation.
There is no n

In [15]:
# Q&A chain
QA_chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff", # other options: map_reduce, refine, etc.
    retriever=retriever,
    return_source_documents=False
    )

In [16]:
# Ask questions
query = "What does the paper propose?"
response = QA_chain(query)
print(f"Question: {query}\nAnswer: {response['result']}")

query = "Who are the authors?"
response = QA_chain(query)
print(f"Question: {query}\nAnswer: {response['result']}")

Question: What does the paper propose?
Answer: The paper proposes a new framework for estimating generative models via an adversarial process, in which we simultaneously train two models: a generative model G that captures the data distribution, and a discriminative model Dthat estimates the probability that a sample came from the training data rather than G. The training procedure for Gis to maximize the probability of Dmaking a mistake.
Question: Who are the authors?
Answer: Ian J. Goodfellow, Jean Pouget-Abadie∗, Mehdi Mirza, Bing Xu, David Warde-Farley, Sherjil Ozair†, Aaron Courville, Yoshua Bengio
